In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import gc
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['addingzerototrain', 'fork-of-treebasedapproachdata', 'competitive-data-science-predict-future-sales', 'textfeatures']


In [2]:
COMBINED_DATA_FPATH='DATA_WITH_TXT.h5'

In [3]:
DATA_FPATH = '../input/fork-of-treebasedapproachdata/DATA.hdf'
TEST_LIKE_SALES_FPATH = '../input/addingzerototrain/train_with_zero.hdf'
SALES_FPATH ='../input/competitive-data-science-predict-future-sales/sales_train.csv'
ITEMS_FPATH = '../input/competitive-data-science-predict-future-sales/items.csv'
SHOPS_FPATH = '../input/competitive-data-science-predict-future-sales/shops.csv'
TEST_SALES_FPATH = '../input/competitive-data-science-predict-future-sales/test.csv'
SAMPLE_SUBMISSION_FPATH = '../input/competitive-data-science-predict-future-sales/sample_submission.csv'
TRAINED_MODEL_FPATH = 'trained_model.bin'

TEXT_FEATURE_FPATH = '../input/textfeatures/text_features.h5'

In [4]:
# Load preprocessed data.
X_df = pd.read_hdf(DATA_FPATH, 'X')
y_df = pd.read_hdf(DATA_FPATH, 'y')

sales_df = pd.read_hdf(TEST_LIKE_SALES_FPATH, 'df')

item_text = pd.read_hdf(TEXT_FEATURE_FPATH, 'item_500')
shop_text = pd.read_hdf(TEXT_FEATURE_FPATH, 'shop_50')
category_text = pd.read_hdf(TEXT_FEATURE_FPATH, 'category_60')

In [5]:
y_df[y_df > 20 ]  = 20
y_df[y_df < 0] = 0


In [6]:
gc.collect()


11

In [7]:
item_text = item_text[['item_id'] + ['item_name_text_{}'.format(i) for i in range(50)]]
shop_text = shop_text[['shop_id'] + ['shop_name_text_{}'.format(i) for i in range(10)]]
category_text = category_text[['item_category_id'] + ['item_category_name_text_{}'.format(i) for i in range(10)]]

In [8]:
def add_text_features(df):
    df.reset_index(inplace=True)
    df = pd.merge(df, shop_text, how='left', on='shop_id')
    print('Shop text added')
    gc.collect()
    df = pd.merge(df, category_text, how='left', on='item_category_id')
    print('Category text added')
    gc.collect()
    df = pd.merge(df, item_text, how='left', on='item_id')
    print('Item text added')
    gc.collect()
    df.set_index('index',inplace=True)
    return df


In [ ]:
y_df.to_hdf(COMBINED_DATA_FPATH, 'y')
del y_df

for year in X_df.year.unique():
    X_train_df = X_df[X_df.year == year].copy()
    X_train_df = add_text_features(X_train_df)
    gc.collect()
    train_columns = X_train_df.columns.tolist()
    X_train_df.to_hdf(COMBINED_DATA_FPATH,'X_{}'.format(year))    
    del X_train_df
    gc.collect()

del X_df
gc.collect()

In [ ]:
len(train_columns)

In [ ]:
test_X_df = pd.read_hdf(DATA_FPATH, 'test_X')
test_X_df = add_text_features(test_X_df)
test_X_df = test_X_df[train_columns]
test_X_df.to_hdf(COMBINED_DATA_FPATH, 'X_test',complevel=9)
